# 8th_hometask

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA

## 2. Обучить любую модель классификации на датасете IRIS до применения самописного PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.

### Датасет IRIS

In [2]:
X, y = load_iris(return_X_y=True)

X.shape, y.shape

((150, 4), (150,))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=True, random_state=0, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 4), (38, 4), (112,), (38,))

### Дерево-классификатор

In [4]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [5]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
            
        # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction

In [6]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [7]:
# Расчет прироста

def gain(left_labels, right_labels, root_gini):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [8]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [9]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
#     min_samples_leaf = 5

    root_gini = gini(labels)

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
#             if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
#                 continue
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [10]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)

#     print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_labels)
    
#     print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [11]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [12]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [13]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

### Обучение и прогноз без PCA

In [14]:
# Построим дерево по обучающей выборке
my_tree = build_tree(X_train, y_train)

In [15]:
# Получим ответы для обучающей выборки 
y_pred_train = predict(X_train, my_tree)

In [16]:
accuracy_metric(y_train, y_pred_train)

100.0

In [17]:
# И получим ответы для тестовой выборки
y_pred_test = predict(X_test, my_tree)

In [18]:
accuracy_metric(y_test, y_pred_test)

94.73684210526315

#### Хорошая точность прогноза на обоих выборках, переобучение.

### Самописный PCA

In [19]:
def my_PCA(X, n_components):
    if not type(X) == np.ndarray:
        raise Exception('X must be numpy.ndarray type!')
    elif not type(n_components) == int:
        raise Exception('n_components must be integer type!')
    elif not 0 < n_components < X.shape[1]:
        raise Exception(f'n_components value must be from one and up to number of columns of matrix X: {X.shape[1]}!')
    
    # Выбор необходимого количества опорных векторов, для получения матрицы перехода.
    eig_vectors = np.linalg.eig(X.T @ X)[1][0:n_components]
    
    # Перевод матрицы объектов-признаков в новый базис.
    return X @ eig_vectors.T

### Датасет IRIS c применением PCA (2 компоненты)

In [20]:
n = 2

X_PCA_2 = my_PCA(X.copy(), n)

X_PCA_2.shape

(150, 2)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_PCA_2, y, train_size=0.75, shuffle=True, random_state=0, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 2), (38, 2), (112,), (38,))

### Обучение и прогноз с PCA (2 компоненты)

In [22]:
# Построим дерево по обучающей выборке
my_tree = build_tree(X_train, y_train)

In [23]:
# Получим ответы для обучающей выборки 
y_pred_train = predict(X_train, my_tree)

In [24]:
accuracy_metric(y_train, y_pred_train)

100.0

In [25]:
# И получим ответы для тестовой выборки
y_pred_test = predict(X_test, my_tree)

In [26]:
accuracy_metric(y_test, y_pred_test)

97.36842105263158

#### На отложенной выборке метрика качества улучшилась после применения метода PCA. Возможно, удалённые признаки содержали шумы, снижающие качество обучения модели. Попробуем проанализировать собственные значения матрицы объектов-признаков X.

In [27]:
eig_values = np.linalg.eig(X.T @ X)[0]
eig_values

array([9.20830507e+03, 3.15454317e+02, 1.19780429e+01, 3.55257020e+00])

In [28]:
eig_values[0] / sum(eig_values) * 100, eig_values[1] / sum(eig_values) * 100

(96.53029806531568, 3.3068951313646835)

#### Первая компонента объясняет более 96.5% дисперсии всей выборки признаков, что является достаточной информативностью для работы модели. Вторая компонента добавляет ещё 3.3%, что суммарно покрывает более 99.8% дисперсии признаков объектов.
#### В данном случае применение метода PCA позволило сократить размерность матрицы объектов-признаков в два раза без значимой потери информативности.
#### Попробуем рассмотреть только одну компоненту.

### Датасет IRIS c применением PCA (1 компонента)

In [29]:
n = 1

X_PCA_1 = my_PCA(X.copy(), n)

X_PCA_1.shape

(150, 1)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_PCA_1, y, train_size=0.75, shuffle=True, random_state=0, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 1), (38, 1), (112,), (38,))

### Обучение и прогноз с PCA (1 компонента)

In [31]:
# Построим дерево по обучающей выборке
my_tree = build_tree(X_train, y_train)

In [32]:
# Получим ответы для обучающей выборки 
y_pred_train = predict(X_train, my_tree)

In [33]:
accuracy_metric(y_train, y_pred_train)

100.0

In [34]:
# И получим ответы для тестовой выборки
y_pred_test = predict(X_test, my_tree)

In [35]:
accuracy_metric(y_test, y_pred_test)

84.21052631578947

#### Метрика качества на отложенной выборке ухудшилась, значит вторая компонента содержит значимую информацию.

## 2. *Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

### Самописный PCA SVG

In [36]:
def my_PCA_SVG(X, n_components):
    if not type(X) == np.ndarray:
        raise Exception('X must be numpy.ndarray type!')
    elif not type(n_components) == int:
        raise Exception('n_components must be integer type!')
    elif not 0 < n_components < X.shape[1]:
        raise Exception(f'n_components value must be from one and up to number of columns of matrix X: {X.shape[1]}!')
    
    # Вычисление матрицы перехода.
    W = np.linalg.svd(X.T @ X)[2].T[:, 0:n_components]
    
    # Перевод матрицы объектов-признаков в новый базис.
    return X @ W

### Проверка
#### Сравним матрицы PCA и PCA SVD.

In [37]:
X_PCA_SVG_2 = my_PCA_SVG(X, 2)

In [38]:
X_PCA_SVG_2[0:5, :]

array([[-5.91274714, -2.30203322],
       [-5.57248242, -1.97182599],
       [-5.44697714, -2.09520636],
       [-5.43645948, -1.87038151],
       [-5.87564494, -2.32829018]])

In [39]:
X_PCA_2[0:5, :]

array([[5.59244325, 2.84325337],
       [5.30013417, 2.49386389],
       [5.15653204, 2.59471988],
       [5.15343468, 2.36698815],
       [5.54574992, 2.8599192 ]])

#### Первые пять строк отличаются.
#### Попробуем обучить модель на новой матрице PCA SVD, чтобы понять, сохранилась ли основная тенденция в данных.

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_PCA_SVG_2, y, train_size=0.75, shuffle=True, random_state=0, stratify=y)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((112, 2), (38, 2), (112,), (38,))

In [41]:
# Построим дерево по обучающей выборке
my_tree = build_tree(X_train, y_train)

In [42]:
# Получим ответы для обучающей выборки 
y_pred_train = predict(X_train, my_tree)

In [43]:
accuracy_metric(y_train, y_pred_train)

100.0

In [44]:
# И получим ответы для тестовой выборки
y_pred_test = predict(X_test, my_tree)

In [45]:
accuracy_metric(y_test, y_pred_test)

100.0

#### Метрика качества на отложенной выборке показала максимальный результат. Подозрительно, но работает.